#Tool updated May 14 2024
##Info
<!-- 

To run this notebook, click menu Cell -> Run All


 -->

In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Model_GIS_Export_Variables import *
import subprocess
import sqlite3
import shutil
from datetime import datetime as dt, timedelta

In [2]:
#Permanent cell 2
#Import RAWN

columns = ['Key','SewerageArea', 'Acronym', 'MHName', 'FacilityID']
for rawn_year in rawn_years:
    columns.append('PWWF_' + str(rawn_year))
rawn_output_df = pd.DataFrame(columns=columns)
rawn_output_df.set_index('Key',inplace=True)

if rawn_input_from_model:
    #How many to rows to skip for this spreadsheet type
    rawn_input_skiprows = 1
    #The PWWF column name in this spreadsheet type
    rawn_pwwf_column = 'PWWF (L/s)'
else:
    rawn_input_skiprows = 13
    rawn_pwwf_column = 'P.W.W.F.'

rawn_list = []

#This csv files contains matched facilityID to MUID, where a match was found in tool 1.
node_df = pd.read_csv(model_manhole_csv, dtype={'facilityid': str,'muid': str})

rawn_list = []
if not rawn_input_from_model: #Read csv file
    rawn_input_df = pd.read_csv(rawn_csv)
else: #create dataframe similar to the rawn csv file by looping through the RAWN Excel file names
    for rawn_inputfolder in rawn_inputfolders:
        model_area = rawn_inputfolder[0]
        folder = rawn_inputfolder[1]
        for f in os.listdir(folder):
            if f[-5:]=='.xlsx':
                filepath = folder + '\\' + f
                comb_name =  os.path.splitext(f)[0]
                acronym = comb_name.split('_')[0]
                mh_name = comb_name.split('_')[1]
                rawn_list.append([model_area,acronym,mh_name,"Sheet1",filepath])
    rawn_input_df = pd.DataFrame(rawn_list,columns=['Sewer_Area', 'Acronym', 'MH_Name', 'Tab', 'Sheet_Path'])
                                   

for index1, row1 in rawn_input_df.iterrows():
    if rawn_input_from_model:
        #This type has column 'NODE' which i set to string.
        rawn_single_df = pd.read_excel(row1['Sheet_Path'],sheet_name=row1['Tab'],skiprows=rawn_input_skiprows,
                                       dtype={'NODE': str})
        #This type has an empty row to be removed, may improve in later version
        rawn_single_df.dropna(how='all', inplace=True)
        #Reset the index to start at 0 
        rawn_single_df.reset_index(inplace=True,drop=True)
    else:
        rawn_single_df = pd.read_excel(row1['Sheet_Path'],sheet_name=row1['Tab'],skiprows=rawn_input_skiprows)
    sewer_area = row1['Sewer_Area']
    acronym = row1['Acronym']
    mh_name = row1['MH_Name']
    index_val = acronym + '_' + mh_name
    if rawn_input_from_model:
        #The muid are in all rows, the below just takes the one from the first row. 
        muid = rawn_single_df.loc[0,'NODE']
        node_match_df = node_df.loc[(node_df.muid==muid) & (node_df.sewer_area)]
    else:
        node_match_df = node_df.loc[(node_df.acronym==acronym) & (node_df.mhname==mh_name) & (node_df.sewer_area)]
    node_match_df.reset_index(inplace=True)
    if len(node_match_df) == 0:
        print('Warning: No FacilityID found for ' + mh_name + ' in ' + acronym)
        facilityid = 'Not Found'
    else:
        facilityid = node_match_df.loc[0,'facilityid']
        
    rawn_output_df.loc[index_val,['SewerageArea','Acronym','MHName','FacilityID']]=[sewer_area,acronym,mh_name,facilityid]
    
    
    #For traditional rawn sheets, the first 4 columns have headers one higher up than the following columns 
    #(which are under merged cells) so this adjustment below are only for traditional RAWN sheets.
    if not rawn_input_from_model:
        #In this version, the column names are in different rows, this is corrected below.
        for i in range(4):
            col_name = rawn_single_df.columns[i]
            #Transfer the column name to the first row to be consistent with other columns
            rawn_single_df.loc[0,col_name]=col_name
        for col in rawn_single_df.columns:
            #Set the column names to equal the value in the first row
            rawn_single_df.rename(columns={col:rawn_single_df.loc[0,col]},inplace=True)
        #Drop the first row which previously had the column names.
        rawn_single_df.drop([0,1],inplace=True)
    for index2, row2 in rawn_single_df.iterrows():
        year = int(row2['YEAR'])
        pwwf = row2[rawn_pwwf_column]
        if year in rawn_years:
            #Input in columns named after the year. This will create the column when it does not exist, otherwise just fill.
            rawn_output_df.loc[index_val,'PWWF_' + str(year)] = pwwf
            
if len(acronym_filter) > 0:
    #If only some acronyms are to be created
    rawn_output_df = rawn_output_df[rawn_output_df.Acronym.isin(acronym_filter)]

#Create the output csv
rawn_output_df.to_csv(output_folder + '\\RAWN_Nodes.csv', index=False)   

In [3]:
#Import model results
for m_index, m in enumerate(master_list):
  
    model_area = m[0]
    model = m[1]
    model_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
    
    db_type = os.path.splitext(model)[1][1:]
    
    node_df = pd.read_csv(model_manhole_csv, dtype={'facilityid': str,'muid': str})
    node_df.rename(columns={'sewer_area':'SewerageArea','facilityid':'FacilityID','muid':'ModelID','match_code':'Match_Code','acronym':'Acronym','mhname':'MHName'},inplace=True)
    node_df = node_df[node_df.SewerageArea==model_area]
    
    pipe_df = pd.read_csv(model_pipe_csv, dtype={'facilityid': str,'muid': str})
    pipe_df.rename(columns={'sewer_area':'SewerageArea','facilityid':'FacilityID','muid':'ModelID','match_code':'Match_Code','acronym':'Acronym'},inplace=True)
    pipe_df = pipe_df[['SewerageArea','FacilityID','Acronym','ModelID']]
    pipe_df = pipe_df[pipe_df.SewerageArea==model_area]

    for r in result_list:
        description = r[0]
        pop_year = r[1]
        result_file = r[2]
        #If mdb (VSA), the results are in the same folder as the model
        if db_type.lower() == 'mdb':
            result_path = model_folder + '\\' + result_file
            if not os.path.exists(result_path):
                #This will end the script execution with an error.
                raise ValueError("The following result file was not found in the model folder: " + result_file)     
        #If sqlite (MIKE+: FSA, NSSA, LISA), then the results are found in sub folders auto-named after the model.
        elif db_type.lower() == 'sqlite':
            file_found = False
            for f1 in os.listdir(model_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder
                    result_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                    for f2 in os.listdir(model_folder + '\\' + result_subfolder):
                        if os.path.basename(f2) == result_file:
                            result_path = model_folder + '\\' + result_subfolder + '\\' + f2
                            file_found = True
            if not file_found:
                #This will end the script execution with an error.
                raise ValueError("The following result file was not found: " + result_file)
        else:
            #This will end the script execution with an error.
            raise ValueError("The variable 'db_type' must be 'mdb' or 'sqlite'.")
 
        res1d = Res1D(result_path)
        print('process ' + result_file)
        sim_start = res1d.time_index.min()
        start = sim_start + timedelta(days=1)
        end = res1d.time_index.max()
        sim_seconds = (end - sim_start).total_seconds()
        #Number of timesteps
        timesteps = len(res1d.time_index)-1
        #Number of seconds in one timestep
        timestep_seconds = sim_seconds / timesteps
        #Number of timesteps in one day (Used to crop first day and for ADWF only include last day)
        one_day_steps = int(86400 / timestep_seconds)
        
        #@@@@@@@@@@@@@@@@@@@@@HGL
        if description.lower() == 'dwf':
            #Build column names and prefill with empty (nan) values.
            hgl_name = 'PDWF_' + str(pop_year) + '_HGL'
            hgl_avg_name = 'ADWF_' + str(pop_year) + '_HGL'
            node_df[hgl_avg_name] = np.nan
            hgl_min_name = 'MDWF_' + str(pop_year) + '_HGL'
            node_df[hgl_min_name] = np.nan
        else:
            hgl_name = description + '_' + str(pop_year) + '_HGL'
        node_df[hgl_name] = np.nan
        nodes = [node.Id for node in res1d.data.Nodes]
        for index, row in node_df.iterrows():
            muid = row['ModelID']
            if muid in nodes:
                #List hgl values with first day removed.
                hgl = max(list(res1d.query.GetNodeValues(muid, "WaterLevel"))[one_day_steps:])
                node_df.loc[index,hgl_name] = hgl
                if description.lower() == 'dwf':
                    #For average, use only last day to calculate average
                    hgl_avg = sum(list(res1d.query.GetNodeValues(muid, "WaterLevel"))[-one_day_steps:])/one_day_steps
                    node_df.loc[index,hgl_avg_name] = hgl_avg
                    hgl_min = min(list(res1d.query.GetNodeValues(muid, "WaterLevel"))[one_day_steps:])
                    node_df.loc[index,hgl_min_name] = hgl_min
                                    
                
        #@@@@@@@@@@@@@@@@@@@@@ Q and V
        #The pipe muids are given a suffix (hyphen and incrementing counter) in mikeio1d which is removed here
        pipes = [pipe.Id[:pipe.Id.rfind('-')] for pipe in res1d.data.Reaches]
        if description.lower() == 'dwf':
            #Build column names and prefill with empty (nan) values.
            q_name = 'PDWF_' + str(pop_year) + '_Q'
            q_avg_name = 'ADWF_' + str(pop_year) + '_Q'
            pipe_df[q_avg_name] = np.nan           
            q_min_name = 'MDWF_' + str(pop_year) + '_Q'
            pipe_df[q_min_name] = np.nan
            
            v_name = 'PDWF_' + str(pop_year) + '_V'
            v_avg_name = 'ADWF_' + str(pop_year) + '_V'
            pipe_df[v_avg_name] = np.nan
            v_min_name = 'MDWF_' + str(pop_year) + '_V'
            pipe_df[v_min_name] = np.nan
            
        else:
            q_name = description + '_' + str(pop_year) + '_Q'
            v_name = description + '_' + str(pop_year) + '_V'
        pipe_df[q_name] = np.nan
        pipe_df[v_name] = np.nan
        for index, row in pipe_df.iterrows():
            muid = row['ModelID']
            if muid in pipes:
                #List of flow values with first day cropped/
                q_list = list(res1d.query.GetReachStartValues(muid, "Discharge"))[one_day_steps:]
                #Convert to L/s
                q_list = [q*1000 for q in q_list]
                v_list = list(res1d.query.GetReachStartValues(muid, "FlowVelocity"))[one_day_steps:]
                
                if absolute_velocity_discharge:
                    #Get the absolute maxes and mins (every value absolute)
                    q = max([abs(q) for q in q_list])
                    v = max([abs(v) for v in v_list])
                    q_min = min([abs(q) for q in q_list])
                    v_min = min([abs(v) for v in v_list])
                    
                else:
                    q = max(q_list)
                    v = max(v_list)
                    q_min = min(q_list)
                    v_min = min(v_list)
                    
                pipe_df.loc[index,q_name] = q
                pipe_df.loc[index,v_name] = v
                
                if description.lower() == 'dwf':
                    pipe_df.loc[index,q_min_name] = q_min
                    pipe_df.loc[index,v_min_name] = v_min
                    
                    q_avg = sum(list(res1d.query.GetReachStartValues(muid, "Discharge"))[-one_day_steps:])/one_day_steps*1000
                    if absolute_velocity_discharge:
                        #Only take the absolute of the final average, not every timestep, otherwise it will be skewed.
                        q_avg = abs(q_avg)
                    pipe_df.loc[index,q_avg_name] = q_avg
                    
                    v_avg = sum(list(res1d.query.GetReachStartValues(muid, "FlowVelocity"))[-one_day_steps:])/one_day_steps
                    if absolute_velocity_discharge:
                        #Only take the absolute of the final average, not every timestep, otherwise it will be skewed.
                        v_avg = abs(v_avg)
                    pipe_df.loc[index,v_avg_name] = v_avg
                    

    if m_index == 0:
        #Create the table with all values
        node_df_all = node_df.copy()
        pipe_df_all = pipe_df.copy()
    else:
        #Append to the table with all value (alread created at m_index == 0)
        node_df_all = pd.concat([node_df_all,node_df])
        pipe_df_all = pd.concat([pipe_df_all,pipe_df])
    
if len(acronym_filter) > 0:
    #If only some acronyms are to be created
    node_df_all = node_df_all[node_df_all.Acronym.isin(acronym_filter)]
    pipe_df_all = pipe_df_all[pipe_df_all.Acronym.isin(acronym_filter)]
    
node_df_all.drop(columns=['Match_Code'],inplace=True)

#Replace column name hyphens with underscores as requested by GIS
node_df_all.rename(columns=lambda x: x.replace('-', '_'), inplace=True)
pipe_df_all.rename(columns=lambda x: x.replace('-', '_'), inplace=True)

#Create the csv files
node_df_all.to_csv(output_folder + '\\Model_Nodes.csv', index=False)   
pipe_df_all.to_csv(output_folder + '\\Model_Pipes.csv', index=False) 
print('Done')                
                



process FSA_DWF_2021-07-22_4d_2025pop_BaseDefault_Network_HD.res1d
process FSA_GA_EX-2y-24h-AES_2025p_Base-DSS1Default_Network_HD.res1d
process FSA_GA_EX-5y-24h-AES_2025p_Base-DSS2Default_Network_HD.res1d
process FSA_GA_EX-10y-24h-AES_2025p_Base-DSS3Default_Network_HD.res1d
process FSA_GA_EX-25y-24h-AES_2025p_Base-DSS16Default_Network_HD.res1d
process FSA_DWF_2021-07-22_4d_2030pop_2030_NetworkDefault_Network_HD.res1d
process FSA_GA_EX-2y-24h-AES_2030p_F_2030_Network-DSS4Default_Network_HD.res1d
process FSA_GA_EX-5y-24h-AES_2030p_F_2030_Network-DSS5Default_Network_HD.res1d
process FSA_GA_EX-10y-24h-AES_2030p_F_2030_Network-DSS6Default_Network_HD.res1d
process FSA_GA_EX-25y-24h-AES_2030p_F_2030_Network-DSS17Default_Network_HD.res1d
process NSSA_DWF_2018-07-26_4d_2025pop_BaseDefault_Network_HD.res1d
process NSSA_GA_EX-2y-24h-AES_2025p_Base-DSS1Default_Network_HD.res1d
process NSSA_GA_EX-5y-24h-AES_2025p_Base-DSS2Default_Network_HD.res1d
process NSSA_GA_EX-10y-24h-AES_2025p_Base-DSS3Defaul